In [22]:
from models.mlp import MLP

In [23]:
model = MLP(10,2,[5,5])

model.state_dict()

OrderedDict([('model.0.0.weight',
              tensor([[ 8.5733e-02,  2.1495e-01,  2.6989e-01,  2.5647e-01, -2.3429e-01,
                       -5.0665e-02, -1.8712e-01, -1.8068e-01,  7.5011e-02,  2.9683e-01],
                      [-2.5333e-01,  1.9138e-01, -1.6395e-01, -6.1593e-02,  2.5101e-01,
                       -7.1521e-02,  2.8794e-02, -2.2105e-01,  2.6919e-01, -4.0879e-02],
                      [-2.3129e-01,  9.2595e-02, -2.2484e-01, -2.5093e-01,  1.9264e-02,
                        2.5072e-01, -8.9487e-02,  1.4884e-01,  2.7174e-01,  2.0974e-01],
                      [-1.6588e-01, -3.4665e-02, -9.9512e-02, -2.5429e-01,  1.3066e-04,
                        2.3794e-01,  2.6643e-01,  2.9504e-02,  7.1823e-02, -1.3690e-01],
                      [ 2.3870e-01, -1.3159e-01, -2.1967e-01,  4.8697e-02,  1.8954e-01,
                       -2.8511e-01,  2.8586e-01,  1.1376e-01, -2.2155e-01, -6.8118e-02]],
                     device='cuda:0')),
             ('model.0.0.bias',
        

In [24]:
import torch
x = torch.randn((10,10))
y = torch.randint(0,2,(10,))

In [25]:

def somefunc(model):
    old_state = model.state_dict()
    model.fit(x,y)
    new_state = model.state_dict()
    model.load_state_dict(old_state)
    return model,new_state


model,n = somefunc(model)

In [26]:
n

OrderedDict([('model.0.0.weight',
              tensor([[ 0.0957,  0.2050,  0.2799,  0.2665, -0.2443, -0.0607, -0.1971, -0.1907,
                        0.0650,  0.2868],
                      [-0.2633,  0.1814, -0.1740, -0.0716,  0.2610, -0.0815,  0.0388, -0.2311,
                        0.2592, -0.0509],
                      [-0.2413,  0.1026, -0.2148, -0.2409,  0.0093,  0.2407, -0.0795,  0.1588,
                        0.2817,  0.1997],
                      [-0.1559, -0.0247, -0.0895, -0.2443, -0.0099,  0.2479,  0.2564,  0.0395,
                        0.0818, -0.1269],
                      [ 0.2487, -0.1416, -0.2297,  0.0587,  0.1795, -0.2751,  0.2759,  0.1038,
                       -0.2116, -0.0781]], device='cuda:0')),
             ('model.0.0.bias',
              tensor([ 0.2644, -0.2326, -0.1024,  0.0937, -0.2670], device='cuda:0')),
             ('model.1.0.weight',
              tensor([[ 0.2552,  0.2328,  0.2674, -0.2851,  0.4454],
                      [-0.2651,  0.4565

In [27]:
model.state_dict()

OrderedDict([('model.0.0.weight',
              tensor([[ 0.0957,  0.2050,  0.2799,  0.2665, -0.2443, -0.0607, -0.1971, -0.1907,
                        0.0650,  0.2868],
                      [-0.2633,  0.1814, -0.1740, -0.0716,  0.2610, -0.0815,  0.0388, -0.2311,
                        0.2592, -0.0509],
                      [-0.2413,  0.1026, -0.2148, -0.2409,  0.0093,  0.2407, -0.0795,  0.1588,
                        0.2817,  0.1997],
                      [-0.1559, -0.0247, -0.0895, -0.2443, -0.0099,  0.2479,  0.2564,  0.0395,
                        0.0818, -0.1269],
                      [ 0.2487, -0.1416, -0.2297,  0.0587,  0.1795, -0.2751,  0.2759,  0.1038,
                       -0.2116, -0.0781]], device='cuda:0')),
             ('model.0.0.bias',
              tensor([ 0.2644, -0.2326, -0.1024,  0.0937, -0.2670], device='cuda:0')),
             ('model.1.0.weight',
              tensor([[ 0.2552,  0.2328,  0.2674, -0.2851,  0.4454],
                      [-0.2651,  0.4565